In [1]:
import scanpy as sc
import pandas as pd
import numpy as np
import networkx as nx
import omnipath, graphtools

/home/aarthivenkat/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [8]:
results = np.load('results/GSPA_QR.npz', allow_pickle=True)
embedding = pd.DataFrame(results['signal_embedding'], index=results['genes'])
df = pd.read_csv('results/gene_summary.csv', index_col=0)

In [ ]:
types = []
for cluster in range(df['gene_clusters'].max() + 1):
    # get genes in gene cluster
    cluster_genes = df[(df['gene_clusters'] == cluster)].index
    
    # subset to genes in gene cluster also in database
    cluster_genes_omnipath = set(cluster_genes).intersection(db_genes)
    cluster_emb = embedding.loc[cluster_genes_omnipath]
    
    #learn KNN graph of genes in gene cluster & in database
    G = nx.Graph(graphtools.Graph(cluster_emb, use_pygsp=True, knn=100).A)
    G = nx.relabel_nodes(G, dict(zip(range(len(cluster_emb)), cluster_genes_omnipath)))
    
    # Keep edges that are also in database
    edges = []
    for row in db[(db['source_genesymbol'].isin(cluster_genes_omnipath)) & (db['target_genesymbol'].isin(cluster_genes_omnipath))].iterrows():
        if G.has_edge(row[1]['source_genesymbol'], row[1]['target_genesymbol']):
            edges.append(f"{row[1]['source_genesymbol']}\t{row[1]['target_genesymbol']}\t{row[1]['type']}")
            types.append(row[1]['type'])
                        
    with open(f'results/Cluster_{cluster}_omnipath_genes.tsv', 'w') as f:
        for edge in edges:
            f.write(f"{edge}\n")